In [1]:
import sys
!{sys.executable} -m pip install --user twitter
from twitter import *

ACCESS_TOKEN = "988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31"
ACCESS_TOKEN_SECRET = "WpcZY8dTZAQVWEJkZBlS8hMQaR9ckqOs8ErFML2x2bVAk"
CONSUMER_KEY = "sRKZDEri6WsOhDmJmxxbnLvyC"
CONSUMER_SECRET = "OJO5n71PvgNE6B6JLePAlRTyOlDoTFFpWziUEE4stu0pbNNLHo"

class twitUser:
    def __init__(self, name, screenName, followerCount, location, degrees, profileImageURL):
        self.name = name
        self.screenName = screenName
        self.followerCount = followerCount
        self.location = location
        self.degrees = degrees
        self.profileImageURL = profileImageURL    
    
def calculateScore(twitUser):
    score = twitUser.followerCount / 3 + 200 / twitUser.degrees
    if twitUser.location == 'Omaha, NE':
        score += 50
    return score

def getFollowers(user, degree, maxFollowers): #recursive function that access followers
    if degree == 3:
        return 0
    x = t.followers.list(screen_name=user, count = 200) #dictionary containing list of followers  
    while True:
        for i in range(0,len(x['users'])): #iterate through list of followers
            follower = x['users'][i] #access follower object
            if follower['followers_count'] > maxFollowers: # don't want users with too many followers b/c unrealistic
                continue
            twitFollower = twitUser(follower['name'], follower['screen_name'], follower['followers_count'], follower['location'], degree, follower['profile_image_url'])
            print(twitFollower.name, calculateScore(twitFollower))
            
            if not follower['protected'] and not follower['verified'] and follower['followers_count'] < 2000: #recursively get follower's followers if account is not private
                getFollowers(follower['screen_name'], degree + 1, maxFollowers)
        cursor = x['next_cursor'] #get next page
        if cursor == 0: #no next page
            break
        x = t.followers.list(screen_name=user, cursor = cursor, count = 200) #turn to next page


# Put in token, token_key, con_secret, con_secret_key
t = Twitter(
    auth=OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))


getFollowers("dijkstra_edsger", 1, 2000)






            



You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Francis Schickel 228.0
Eddie 398.33333333333337
Nathan Reider 204.0
Kelly Crithfield 105.66666666666667
Chris McDonald 116.66666666666667
Matt Kirkle 204.0
RJ 309.33333333333337
JerryFanPage 107.33333333333333
Brandon Cohen 112.66666666666667
Courtney Riedmann 307.33333333333337
cris aleman 163.66666666666669
Andy Elsaesser 123.0
John Dechant 199.33333333333334
Steven Winston 105.0
Kyle White 191.33333333333331
Cole Demro 169.0
Nico Robalin 196.0
džana🍍 487.0
Jerry 118.66666666666667
Mary 102.66666666666667
Dylan Delgado 104.33333333333333
Kimberlynne 286.66666666666663
kay nigh 379.3333333333333
Megha Devaraj 144.66666666666666
Luke Kroll 209.0
Jeff DeHaven 234.66666666666669
Thomas Gerot 110.66666666666667
Andrew Goff 308.66666666666663
Andrew Fuller 165.0
Edwin 162.0
Marcos Salamanca 164.0
Lauren Hanson 186.0
well i mean i guess 119.0
Jon P

Ryan Reeves 238.33333333333331
Nate LaFave 222.0
Nolan Chaloupka© 210.33333333333331
Omar Nazarie 268.6666666666667
Calvin Rudolph 312.66666666666663
jayden harpenau 281.0
Charlee Sharp 337.33333333333337
Childish Bambino 182.33333333333334
Ali 305.66666666666663
Lucas Felger 130.0
RyGuy 213.33333333333331
Ky 249.0
Justin Mock 211.0
Logan Skrabal 192.0
Shelby Dergan 251.0
Carly O'Keefe 362.0
Dylan Christiansen 163.0
cassidyloberg 349.66666666666663
michael 101.33333333333333
Jake 218.0
callie 245.33333333333331
TK 272.0
Amanda Gekas 298.66666666666663
Nathan Hawkinson 209.33333333333331
Alec 129.33333333333334
Ryan Emmel 141.33333333333334
Sarah Petersen 335.33333333333337
Alec White 110.33333333333333
Leonard Zhang 101.33333333333333
gkelly 268.0
Seth house 115.0
han 379.33333333333337
aly b 340.33333333333337
Olivia Bickmore 328.33333333333337
rachel 419.3333333333333
emily zhang 237.0
Pierce Lam 207.0
Drew Firkins 509.3333333333333
Shirley 118.33333333333333
Marcos Salamanca 264.0
J